# Planning and PDDL

This notebook examines the use of *planning* in artifical intelligence.  We will look at a classical planning algorithm called STRIPS (Standford Research Institute Problem Solver), which uses a language for defining plans called PDDL (Planning Domain Definition Language).  STRIPS, PDDL and classic planning are all defined in the Russell and Norvig textbook AI A Modern Approach, in chapter 11.  I also recommend this tutorial by Kory Becker: [Artificial Intelligence Planning with STRIPS, A Gentle Introduction](http://www.primaryobjects.com/2015/11/06/artificial-intelligence-planning-with-strips-a-gentle-introduction/).  She has an [online demo](https://stripsfiddle.herokuapp.com) that lets you upload your own PDDL-specified problems, and find solutions to them.  


We will use this Python library:
* [PDDL Parser](https://github.com/pucrs-automated-planning/pddl-parser)

In PDDL, we can define **action schema** to represent actions.  Here is an example of an action schema for flying a plane from one location to another: 
```
(:action fly
     :parameters (?p - plane ?from - airport ?to - airport)
     :precondition (and (plane ?p) (airport ?from) (airport ?to) (at ?p ?from))
     :effect (and (at ?p ?to)) (not (at ?p ?from)))
)
```

This defines an action called **fly**, which takes 3 arguments: a plane **p**, a starting airport **from** and a destination airport **to**.  In order for this action to be applied, several *preonditions* must be satisified:
1. **p** must be a plane
1. **from** must be be an airport
1. **to** must be be an airport
1. **p** must initially be located at **from**.



Once the action is applied, then it has the *effect* of changing several states in the world.
1. **p** is no longer located at **from** 
1. **p** is now located at **to** 

In general, a schema consists of:
* an action name
* a list of all the variables use in the schema
* a precondition - a conjunction of literals (positive or negated atomic logical sentences)
* an effect - a conjunction of literals

_Note: The enforcement of argument types like `(plane ?p)` can be left out of the action schema if we specify types in the domain PDDL file._



## What to do 

In this exercise, you'll be writing PDDL for action castles, and then using the PDDL Parser and its planner to come up with solutions to different challenges in the game.

To do this, you'll create pairs of files.  One file will represent the __domain__ and the other file will represent the __problem__. 

The domain file will specify the __types__ of things in your game, and a set of possible __actions__.

The problem file will specify an instance of the game by giving its starting state.  For example, it will contain the list of how locations are connected, and where all of the objects are located at the start of the game.


In [ ]:
# Install the PDDL parser
!git clone https://github.com/pucrs-automated-planning/pddl-parser
!cd pddl-parser

Cloning into 'pddl-parser'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 334 (delta 36), reused 55 (delta 22), pack-reused 260
Receiving objects: 100% (334/334), 78.75 KiB | 15.75 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [ ]:
# Mount your Google Drive so that you can save your PDDL files.
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/CIS 700/planning'


Mounted at /content/drive


In [ ]:
import os

def write_to_file(string, path, filename):
  """This is a helper function to create a file from a string."""
  if not os.path.exists(path) and path != "":
    os.makedirs(path)
  with open(os.path.join(path, filename), "w") as f:
      f.write(string)

## Example Domain
Here's a starting example of a PDDL for the Action Castle domain.  

```
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )
)
```
This specifies 5 types of things that are in the game:
* the player
* locations
* directions
* monsters (like the troll, the guard and the ghost)
* items

We also define one action to get you started.  The action is `go`.  It takes in 4 arguments:
* the direction to travel in
* the player to move
* the starting location
* the destination location

`Precondition` specifies what must be true in order to apply the `go` action, and `effect` specifies how the state of the world will change as a result of applying it.


In [ ]:
domain_1 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )
)
"""

# Write the domain to a PDDL file
domain_filename = "domain.pddl"
write_to_file(domain_1, path, domain_filename)

## Example Problem 

Here's an example of how to specify a problem called `navigate-to-location` in the `action-castle` domain.  We'll specify thee objects, which are instances of the types.  (We'll call our player the `npc` which stands for "non-playable character", to indicate that the AI is controling it, rather than us).  We'll also list out all of the locations and directions in the game.

We can introduce predicates like
```
(connected cottage out gardenpath)
```
or 
```
(at npc cottage)
```
that specify the starting conditions of the game.  These can be "fluent" meaning that they can change over the course of the game by application of the actions. 

In [ ]:
problem_1 = """
(define (problem navigate-to-location)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
   )

   (:goal (and (at npc throneroom)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem.pddl"
write_to_file(problem_1, path, problem_filename)

## Check that your files are correctly formatted

You can check that your PDDL files are correctly formatted by running this command.  It will print out all of the parts of your PDDL files if they are correctly formatted, or it will thrown an Exception if you have a formatting error.

_Note: The PDDL notation goes back to the LISP programming language, which was a popular language in the early days of AI, when STRIPS was being developed.  It takes some getting used to, but hopefully it won't be too difficult for you to understand the format._

In [ ]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'monster', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]]]
----------------------------
['define',
 ['problem', 'navigate-to-location'],
 [':domain', 'action-castle'],
 [':objects',
  'npc',
  '-',
  'player',
  'cottage',
  'gardenpath',
  'fishingpond',
  'gardenpath',
  'windingpath',
  'talltree',
  'drawbridge',
  'courtyard',
  'towerstairs',
  'tower',
  'dungeonstairs',
  'dungeon',
  'greatfeastinghall',
  'throneroom',
  '-',
  'location',
  'in',
  'out',
  'north',
  'south',


## Compute a plan

You can use the planner in the PDDL Parser package to create a plan. The plan is a sequence of actions that will take you from the start state (specified in your problem.pddl file) to the goal (specified in the same file).

In [ ]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.026633739471435547s
plan:
go out npc cottage gardenpath
go north npc gardenpath windingpath
go east npc windingpath drawbridge
go east npc drawbridge courtyard
go east npc courtyard greatfeastinghall
go east npc greatfeastinghall throneroom


# TODO: Move Objects to Desired Location

__Domain__: 
Add two new actions to our action-castle domain PDDL:
* `get`
* `drop`

__Problem__:
Create several items like the fishing pole, the rose, the crown, and put them in their starting locaiton in your problem PDDL.  Set the goal to be `(:goal (and (at crown throneroom)))`.

Check to see if the NPC can move the crown to the throne room.



In [ ]:
domain_2 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types 
      player location direction item
   )

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (guarded ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action drop
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (inventory ?p ?item))
      :effect (and (at ?item ?l1) (not (inventory ?p ?item)))
   )
)
"""

# Write the domain to a PDDL file
domain_filename = "domain-2-move-objects.pddl"
write_to_file(domain_1, path, domain_filename)

In [ ]:
problem_2 = """
(define (problem navigate-to-location)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      TODO - add the item names here
   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      TODO - add thestarting locations for items here
   )

    
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-2-move-objects.pddl"
write_to_file(problem_2, path, problem_filename)

### Check your file formats

If you run this code without updating the domain and problem PDDL files, you'll get an error that says `AttributeError: 'str' object has no attribute 'pop'`.  That indicates that your PDDL format is incorrect.  

In [ ]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

### Compute a plan

In [ ]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.002635955810546875s
plan:


# TODO: Go Fishing


__Domain Updates__: 
Add a new action schema called `gofish` that requires the fishing pole to catch a fish at a location that has a pond.


__Problem Updates__:
Add a new predicate that indicates which locations where fishing can take place.  
```
(haslake fishingpond)
```
Set the goal to be `(:goal (and (inventory npc fish)))`.



_Hints: 
 It might be helpful to create new types in your domain and problem specification like: fishingpole, and food.  This might help avoid being able to catch a fish with a rose._

In [ ]:
domain_3 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types 
      player location direction item
   )

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (guarded ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action drop
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (inventory ?p ?item))
      :effect (and (at ?item ?l1) (not (inventory ?p ?item)))
   )

   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )
)
"""

problem_3 = """
(define (problem go-fish)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      fish rose crown pole - item
   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (at npc cottage)
      (at pole cottage)
      (ispole pole)
      (haslake fishingpond)
      (isfish fish)
      (food fish)
   )

   (:goal (and (inventory npc fish)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-3-fishing.pddl"
write_to_file(problem_3, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-3-fishing.pddl"
write_to_file(domain_3, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'



----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['guarded', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]],
 [':action',
  'get',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['at', '?item', '?l1']],
  ':effect',
  ['and', ['inventory', '?p', '?item'], ['not', ['at', '?item', '?l1']]]],
 [':action',
  'drop',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['inventory', '?p', '?item

In [ ]:
# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.03198647499084473s
plan:
get pole npc cottage
go out npc cottage gardenpath
go south npc gardenpath fishingpond
gofish pole npc fishingpond fish
get fish npc fishingpond


# TODO: Feed the Troll


__Domain Updates__: 
Add a new action schema called `feed` that feeds food to a monster, and results in the monster no longer being hungry, and the food being gone.


__Problem Updates__:
Add a new predicate to the start state.
```
(hungry troll)
```
And a new goal:
```
 (:goal (and (not (hungry troll))))
```




In [ ]:
domain_4 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action drop
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (inventory ?p ?item))
      :effect (and (at ?item ?l1) (not (inventory ?p ?item)))
   )

   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )


   (:action feed
      :parameters (?m - monster ?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?m ?l1) (hungry ?m) (food ?item) (inventory ?p ?item))
      :effect (and (not (hungry ?m)) (not (inventory ?p ?item)))
   )

   (:action unblock-troll
      :parameters (?m - monster ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (istroll ?m) (not (hungry ?m)) (blocked ?l1 ?dir ?l2))
      :effect (not (blocked ?l1 ?dir ?l2))
   )


)

"""

problem_4 = """
(define (problem feed-the-troll)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      pole fish rose crown - item
      troll - monster

   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      (at pole cottage)
      (haslake fishingpond)
      (ispole pole)
      (isfish fish)
      (food fish)
      (at troll drawbridge)
      (hungry troll)

      
   )

   (:goal (and (not (hungry troll))))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-4-feed-the-troll.pddl"
write_to_file(problem_4, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-4-feed-the-troll.pddl"
write_to_file(domain_4, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'monster', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]],
 [':action',
  'get',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['at', '?item', '?l1']],
  ':effect',
  ['and', ['inventory', '?p', '?item'], ['not', ['at', '?item', '?l1']]]],
 [':action',
  'drop',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], 

# Optional: Unblock Troll

Our PDDL naviation example left out blocked locations, which are an important part of the puzzle solving of text adventure games. 

You can you implement a block for the troll, which is resolved after the troll is fed?

```
(blocked drawbridge east courtyard)
(:goal (and (at npc tower)))
```


In [ ]:
domain_5 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )


   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )


   (:action feed-troll
      :parameters (?m - monster ?item - item ?p - player ?l1 - location  ?dir - direction ?l2 - location) 
      :precondition (and (at ?p ?l1) (at ?m ?l1) (hungry ?m) (istroll ?m) (food ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (not (hungry ?m)) (not (inventory ?p ?item)) (not (blocked ?l1 ?dir ?l2)))
   )

   (:action attack-guard
      :parameters (?m - monster ?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (isguard ?m) (at ?p ?l1) (at ?m ?l1) (weapon ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (unconscious ?m) (not (blocked ?l1 ?dir ?l2)))
   )




)

"""

problem_5 = """
(define (problem unblock-troll)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      pole fish rose crown - item
      troll - monster

   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      (at pole cottage)
      (haslake fishingpond)
      (ispole pole)
      (isfish fish)
      (food fish)
      (at troll drawbridge)
      (hungry troll)
      (istroll troll)
      (blocked drawbridge east courtyard)

   )

   (:goal (and (at npc tower)))
)

"""

# Write the problem to a PDDL file
problem_filename = "problem-5-unblock-troll.pddl"
write_to_file(problem_5, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-5-unblock-troll.pddl"
write_to_file(domain_5, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'monster', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]],
 [':action',
  'get',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['at', '?item', '?l1']],
  ':effect',
  ['and', ['inventory', '?p', '?item'], ['not', ['at', '?item', '?l1']]]],
 [':action',
  'gofish',
  ':parameters',
  ['?item',
   '-',
   'item',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?item2',
   '-',
   '

# Optional: Unblock Guard

You can you implement a block for the guard, which is resolved after the you hit the guard with the branch?

```
      (at branch talltree)
      (weapon branch)
      (at guard courtyard)
      (blocked courtyard east greatfeastinghall)

```


In [ ]:
domain_6 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )


   (:action feed-troll
      :parameters (?m - monster ?item - item ?p - player ?l1 - location  ?dir - direction ?l2 - location) 
      :precondition (and (at ?p ?l1) (at ?m ?l1) (hungry ?m) (istroll ?m) (food ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (not (hungry ?m)) (not (inventory ?p ?item)) (not (blocked ?l1 ?dir ?l2)))
   )

   (:action attack-guard
      :parameters (?m - monster ?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (isguard ?m) (at ?p ?l1) (at ?m ?l1) (weapon ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (unconscious ?m) (at key ?l1) (not (blocked ?l1 ?dir ?l2)))
   )


   (:action unlock-tower
      :parameters (?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (iskey ?item) (inventory ?p ?item) (at ?p ?l1) (locked ?l2) (blocked ?l1 ?dir ?l2))
      :effect (and (not (blocked ?l1 ?dir ?l2)))
   )


   (:action give-rose-to-princess
      :parameters (?item - item ?p2 - player ?p1 - player ?l1 - location ) 
      :precondition (and (at ?p1 ?l1) (at ?p2 ?l1) (inventory ?p1 ?item) (isrose ?item))
      :effect (and (inventory ?p2 ?item) (not (inventory ?p1 ?item)))
   )



)

"""

problem_6 = """
(define (problem unblock-guard)
   (:domain action-castle)

   (:objects
      npc - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      pole fish rose crown branch - item
      troll guard - monster

   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      (at pole cottage)
      (haslake fishingpond)
      (ispole pole)
      (isfish fish)
      (food fish)
      (at troll drawbridge)
      (hungry troll)
      (istroll troll)
      (blocked drawbridge east courtyard)

      (at branch talltree)
      (weapon branch)
      (at guard courtyard)
      (isguard guard)
      (blocked courtyard east greatfeastinghall)

   )

   (:goal (and (at npc throneroom)))
)

"""

# Write the problem to a PDDL file
problem_filename = "problem-6-unblock-guard.pddl"
write_to_file(problem_6, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-6-unblock-guard.pddl"
write_to_file(domain_6, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'monster', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]],
 [':action',
  'get',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['at', '?item', '?l1']],
  ':effect',
  ['and', ['inventory', '?p', '?item'], ['not', ['at', '?item', '?l1']]]],
 [':action',
  'gofish',
  ':parameters',
  ['?item',
   '-',
   'item',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?item2',
   '-',
   '

# Optional: Give the Rose to the Princess

The princess is locked in the tower.  Can you find the key and then bring her the rose?


```
      (at rose gardenpath)
      (blocked towerstairs up tower)
      (locked tower)
      (at princess tower)

   (:goal (and (inventory princess rose) (at npc tower)))
``` 

In [ ]:
domain_7 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )


   (:action feed-troll
      :parameters (?m - monster ?item - item ?p - player ?l1 - location  ?dir - direction ?l2 - location) 
      :precondition (and (at ?p ?l1) (at ?m ?l1) (hungry ?m) (istroll ?m) (food ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (not (hungry ?m)) (not (inventory ?p ?item)) (not (blocked ?l1 ?dir ?l2)))
   )

   (:action attack-guard
      :parameters (?m - monster ?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (isguard ?m) (at ?p ?l1) (at ?m ?l1) (weapon ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (unconscious ?m) (at key ?l1) (not (blocked ?l1 ?dir ?l2)))
   )


   (:action unlock-tower
      :parameters (?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (iskey ?item) (inventory ?p ?item) (at ?p ?l1) (locked ?l2) (blocked ?l1 ?dir ?l2))
      :effect (and (not (blocked ?l1 ?dir ?l2)))
   )


   (:action give-rose-to-princess
      :parameters (?item - item ?p2 - player ?p1 - player ?l1 - location ) 
      :precondition (and (at ?p1 ?l1) (at ?p2 ?l1) (inventory ?p1 ?item) (isrose ?item))
      :effect (and (inventory ?p2 ?item) (not (inventory ?p1 ?item)))
   )



)

"""

problem_7 = """
(define (problem unblock-guard)
   (:domain action-castle)

   (:objects
      npc princess - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      pole fish rose crown branch key candle - item
      troll guard  ghost - monster

   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      (at pole cottage)
      (haslake fishingpond)
      (ispole pole)
      (isfish fish)
      (food fish)
      (at troll drawbridge)
      (hungry troll)
      (istroll troll)
      (blocked drawbridge east courtyard)

      (at branch talltree)
      (weapon branch)
      (at guard courtyard)
      (isguard guard)
      (blocked courtyard east greatfeastinghall)


      (at rose gardenpath)
      (blocked towerstairs up tower)
      (locked tower)
      (at princess tower)
      (iskey key)
      (isrose rose)


   )

   (:goal (and (inventory princess rose) (at npc tower)))
)

"""

# Write the problem to a PDDL file
problem_filename = "problem-7-give-rose-to-princess.pddl"
write_to_file(problem_7, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-7-give-rose-to-princess.pddl"
write_to_file(domain_7, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'action-castle'],
 [':requirements', ':strips', ':typing'],
 [':types', 'player', 'location', 'direction', 'monster', 'item'],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at', '?p', '?l1']]]],
 [':action',
  'get',
  ':parameters',
  ['?item', '-', 'item', '?p', '-', 'player', '?l1', '-', 'location'],
  ':precondition',
  ['and', ['at', '?p', '?l1'], ['at', '?item', '?l1']],
  ':effect',
  ['and', ['inventory', '?p', '?item'], ['not', ['at', '?item', '?l1']]]],
 [':action',
  'gofish',
  ':parameters',
  ['?item',
   '-',
   'item',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?item2',
   '-',
   '

# Optional: Get the Crown from the Ghost

The crown is held by the ghost of the king.  In order to get the crown, you'll first need to descend the  stairs to the dungeon (which are too dark to see), then exorcize the ghost by reading the runes on the lit the candle.

```
      (at candle greatfeastinghall)
      (iscandle candle)
      (blocked dungeonstairs down dungeon)
      (dark dungeon)
      (at ghost dungeon)

   (:goal (and (inventory princess crown)))
``` 

In [ ]:
domain_8 = """
(define (domain action-castle)
   (:requirements :strips :typing)
   (:types player location direction monster item)

   (:action go
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get
      :parameters (?item - item ?p - player ?l1 - location ) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action gofish
      :parameters (?item - item ?p - player ?l1 - location ?item2 - item) 
      :precondition (and (at ?p ?l1)  (haslake ?l1) (inventory ?p ?item) (ispole ?item) (isfish ?item2))
      :effect (and (at ?item2 ?l1))
   )


   (:action feed-troll
      :parameters (?m - monster ?item - item ?p - player ?l1 - location  ?dir - direction ?l2 - location) 
      :precondition (and (at ?p ?l1) (at ?m ?l1) (hungry ?m) (istroll ?m) (food ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (not (hungry ?m)) (not (inventory ?p ?item)) (not (blocked ?l1 ?dir ?l2)))
   )

   (:action attack-guard
      :parameters (?m - monster ?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (isguard ?m) (at ?p ?l1) (at ?m ?l1) (weapon ?item) (inventory ?p ?item) (blocked ?l1 ?dir ?l2))
      :effect (and (unconscious ?m) (at key ?l1) (not (blocked ?l1 ?dir ?l2)))
   )


   (:action unlock-tower
      :parameters (?item - item ?p - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (iskey ?item) (inventory ?p ?item) (at ?p ?l1) (locked ?l2) (blocked ?l1 ?dir ?l2))
      :effect (and (not (blocked ?l1 ?dir ?l2)))
   )


   (:action give-rose-to-princess
      :parameters (?item - item ?p2 - player ?p1 - player ?l1 - location ) 
      :precondition (and (at ?p1 ?l1) (at ?p2 ?l1) (inventory ?p1 ?item) (isrose ?item))
      :effect (and (inventory ?p2 ?item) (not (inventory ?p1 ?item)))
   )



   (:action light-candle
      :parameters (?item - item ?p1 - player ?l1 - location ?dir - direction ?l2 - location) 
      :precondition (and (inventory ?p1 ?item) (dark ?l1) (iscandle ?item))
      :effect (and (not (dark ?l1)) (not (blocked ?l1 ?dir ?l2)))
   )



)

"""

problem_8 = """
(define (problem unblock-guard)
   (:domain action-castle)

   (:objects
      npc princess - player
      cottage gardenpath fishingpond gardenpath windingpath talltree drawbridge courtyard towerstairs tower dungeonstairs dungeon greatfeastinghall throneroom - location
      in out north south east west up down - direction
      pole fish rose crown branch key candle - item
      troll guard ghost - monster

   )

   (:init
      (connected cottage out gardenpath)
      (connected gardenpath in cottage)
      (connected gardenpath south fishingpond)
      (connected fishingpond north gardenpath)
      (connected gardenpath north windingpath)
      (connected windingpath south gardenpath)
      (connected windingpath up talltree)
      (connected talltree down windingpath)
      (connected windingpath east drawbridge)
      (connected drawbridge west windingpath)
      (connected drawbridge east courtyard)
      (connected courtyard west drawbridge)
      (connected courtyard up towerstairs)
      (connected towerstairs down courtyard)
      (connected towerstairs up tower)
      (connected tower down towerstairs)
      (connected courtyard down dungeonstairs)
      (connected dungeonstairs up courtyard)
      (connected dungeonstairs down dungeon)
      (connected dungeon up dungeonstairs)
      (connected courtyard east greatfeastinghall)
      (connected greatfeastinghall west courtyard)
      (connected greatfeastinghall east throneroom)
      (connected throneroom west greatfeastinghall)
      (at npc cottage)
      (at pole cottage)
      (haslake fishingpond)
      (ispole pole)
      (isfish fish)
      (food fish)
      (at troll drawbridge)
      (hungry troll)
      (istroll troll)
      (blocked drawbridge east courtyard)


      (at branch talltree)
      (weapon branch)
      (at guard courtyard)
      (isguard guard)
      (blocked courtyard east greatfeastinghall)


      (at rose gardenpath)
      (blocked towerstairs up tower)
      (locked tower)
      (at princess tower)
      (iskey key)
      (isrose rose)


      (at candle greatfeastinghall)
      (iscandle candle)
      (blocked dungeonstairs down dungeon)
      (dark dungeon)
      (at crown dungeon)
      (at ghost dungeon)
   )

   (:goal (and (inventory npc crown)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-8-ghost.pddl"
write_to_file(problem_8, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-8-ghost.pddl"
write_to_file(domain_8, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

In [ ]:
# Optional: Win the Game

Wear the crowd. Sit on the throne. Win the game.

```
   (:goal (and (worn npc crown) (on npc throne)))
``` 

In [ ]:
domain_9 = """
TODO
"""

problem_9 = """
(define (problem sit-on-throne)

TODO 

(:goal (and (word npc crown) (sittingon npc throne)))
"""

# Write the problem to a PDDL file
problem_filename = "problem-9-sit-on-throne.pddl"
write_to_file(problem_9, path, problem_filename)

# Write the domain to a PDDL file
domain_filename = "domain-9-sit-on-throne.pddl"
write_to_file(domain_9, path, domain_filename)

# Check your files are formatted correctly
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

# Compute a plan
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'